In [1]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr

import os

In [2]:
data = pd.read_csv('../../data/sequence-level/data_seq.csv')

In [3]:
data_final = pd.read_csv('../../data/sequence-level/data_seq_final.csv')

In [20]:
# percentage of sequences of minimum length 20
data_final.groupby('sentenceID').filter(lambda x: len(x) >= 20).sentenceID.nunique()/data_final.sentenceID.nunique()

0.8526181081369308

In [4]:
data.shape

(3844150, 14)

In [13]:
data

,userID,userId,timestamp,date,activity,category_name,sequence_number,app_name,sessionID,category_list,seq_length,seq_freq,seq_duration,usID
0,1,01ADD53B,1.511423e+09,2017-11-23 07:44:19.952000000,SCREEN,ON,1.0,NaN,1,"ON,OFF",2,367835.0,0.338,76202
1,1,01ADD53B,1.511423e+09,2017-11-23 07:44:20.289999872,SCREEN,OFF,1.0,NaN,1,"ON,OFF",2,367835.0,0.338,76202
2,1,01ADD53B,1.511424e+09,2017-11-23 08:02:01.137000192,SCREEN,ON,2.0,NaN,2,"ON,OFF",2,367835.0,12.802,76202
3,1,01ADD53B,1.511424e+09,2017-11-23 08:02:13.938999808,SCREEN,OFF,2.0,NaN,2,"ON,OFF",2,367835.0,12.802,76202
4,1,01ADD53B,1.511424e+09,2017-11-23 08:02:51.140999936,SCREEN,ON,3.0,NaN,3,"ON,INBOX,ON,OFF",4,9.0,164.784,16586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3844145,310,FEB02F63,1.515952e+09,2018-01-14 17:53:14.138000128,SCREEN,OFF,1512.0,NaN,844294,"ON,OFF",2,367835.0,9.706,76202
3844146,310,FEB02F63,1.515952e+09,2018-01-14 17:53:32.335000064,SCREEN,ON,1513.0,NaN,844295,"ON,OFF",2,367835.0,9.891,76202
3844147,310,FEB02F63,1.515952e+09,2018-01-14 17:53:42.226000128,SCREEN,OFF,1513.0,NaN,844295,"ON,OFF",2,367835.0,9.891,76202
3844148,310,FEB02F63,1.515953e+09,2018-01-14 17:56:59.348000000,SCREEN,ON,1514.0,NaN,844296,"ON,OFF",2,367835.0,9.538,76202


In [12]:
sum(data_final.usID == 76202)

367835

In [25]:
min_item_support = 5
min_session_length = 2
min_user_sessions = 3
drop_first = True

In [26]:
USER_KEY = 'userID'
TIME_KEY = 'timestamp'
ITEM_KEY = 'usID'
SESSION_KEY = 'sentenceID'

In [27]:
N = 10
data = data[data['seq_freq']>=N].groupby(['sessionID']).first()

In [21]:
start_day = pd.to_datetime(data.timestamp.min(), unit='s').date()
end_day = pd.to_datetime(data.timestamp.max(), unit='s').date()
day_range = pd.date_range(start_day, end_day, freq='D')

# helper list (same length as data) containing the day
user_day = data['userID'].astype(str).str.zfill(3) + '_' + pd.to_datetime(data['timestamp'], unit='s').apply(lambda x: x.date()).astype(str)

sentence_mapping = dict([(y,x+1) for x,y in enumerate(sorted(set(user_day)))])
sentence_indexes = [sentence_mapping[x] for x in user_day]

data['sentenceID'] = sentence_indexes
data['sentence_freq'] = data.groupby([SESSION_KEY])[SESSION_KEY].transform('size')

session_lengths = data.groupby('sentenceID').sentenceID.count()
q1_eps = np.quantile(session_lengths.values, 0.25)
median_eps = np.quantile(session_lengths.values, 0.50)
q3_eps = np.quantile(session_lengths.values, 0.75)

In [22]:
print('For sequence-level analysis, user sequences are tokens (events) and daily concatenations thereof are sentences (sessions).')
print('For sequence-level analysis, only tokens with frequency of at least ' + str(N) + ' were included')
print('The sequence-level data contains a total of:')
print('     - ' + str(data[SESSION_KEY].nunique()) + ' sentences')
print('     - ' + str(sum(data.sentence_freq==1)) + ' sentences with only one token (excluded during data split due to minimum session length of 2)')
print('     - ' + str(data.shape[0]) + ' tokens')
print('     - ' + str(data.usID.nunique()) + ' unique tokens')
print('1st quartile of events per session: ' + str(round(q1_eps, 2)))
print('median number of events per session: ' + str(round(median_eps, 2)))
print('3rd quartile of events per session: ' + str(round(q3_eps, 2)))

For sequence-level analysis, user sequences are tokens (events) and daily concatenations thereof are sentences (sessions).
For sequence-level analysis, only tokens with frequency of at least 10 were included
The sequence-level data contains a total of:
     - 9377 sentences
     - 234 sentences with only one token (excluded during data split due to minimum session length of 2)
     - 720379 tokens
     - 2454 unique tokens
1st quartile of events per session: 34.0
median number of events per session: 66.0
3rd quartile of events per session: 106.0


In [20]:
data.groupby('sentenceID').sentenceID.count()

sentenceID
1        60
2        57
3        12
4        15
5        54
       ... 
9373     12
9374      2
9375     62
9376    105
9377     57
Name: usID, Length: 9377, dtype: int64

In [40]:
print('For sequence-level analysis, we have: ')
for i in [1,2,3,4,5,6,7,8,9,10,15,20,50,100,200,500]:
    print('     - ' + str(sum(data.sentence_freq>=i)) + ' sentences of length >= ' + str(i))
print('Clearly, most of the sentences are not short. Therefore, restricting ourselves to sentences of a certain minimum length, say, 20, would not have much of an impact.')
print('Based on this, language modeling techniques should work reasonably well.')
print('Furthermore, we might expect LM techniques to perform rather well on predicting tokens in higher positions.')

For sequence-level analysis, we have: 
     - 720379 sentences of length >= 1
     - 720145 sentences of length >= 2
     - 719851 sentences of length >= 3
     - 719608 sentences of length >= 4
     - 719376 sentences of length >= 5
     - 719076 sentences of length >= 6
     - 718692 sentences of length >= 7
     - 718342 sentences of length >= 8
     - 717894 sentences of length >= 9
     - 717498 sentences of length >= 10
     - 713946 sentences of length >= 15
     - 708961 sentences of length >= 20
     - 634510 sentences of length >= 50
     - 397801 sentences of length >= 100
     - 93476 sentences of length >= 200
     - 3830 sentences of length >= 500
Clearly, most of the sentences are not short. Therefore, restricting ourselves to sentences of a certain minimum length, say, 20, would not have much of an impact.
Based on this, language modeling techniques should work reasonably well.
Furthermore, we might expect LM techniques to perform rather well on predicting tokens in hig